In [30]:
import nltk
import random
from subprocess import call
import os
os.environ['PATH'] += os.pathsep + '/usr/local/bin'

In [31]:
sentence = "A man and a woman standing in the kitchen"

In [32]:
tokens = nltk.word_tokenize(sentence)
tokens

['A', 'man', 'and', 'a', 'woman', 'standing', 'in', 'the', 'kitchen']

In [33]:
tagged = nltk.pos_tag(tokens)
tagged

[('A', 'DT'),
 ('man', 'NN'),
 ('and', 'CC'),
 ('a', 'DT'),
 ('woman', 'NN'),
 ('standing', 'VBG'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('kitchen', 'NN')]

In [34]:
entities = nltk.chunk.ne_chunk(tagged)

In [35]:
insults = ["ugly", "fat", "smelly","greasy","dirty","stupid as fuck","lame"]

In [36]:
transformed = []
for currentpair in tagged:
    if (currentpair[1]=='NN'): transformed.append(random.choice(insults))
    transformed.append(currentpair[0])
call(["say",str(transformed)])


0

In [37]:
print transformed

['A', 'dirty', 'man', 'and', 'a', 'fat', 'woman', 'standing', 'in', 'the', 'greasy', 'kitchen']


In [38]:
# Used when tokenizing words
sentence_re = r'''(?x)      # set flag to allow verbose regexps
      ([A-Z])(\.[A-Z])+\.?  # abbreviations, e.g. U.S.A.
    | \w+(-\w+)*            # words with optional internal hyphens
    | \$?\d+(\.\d+)?%?      # currency and percentages, e.g. $12.40, 82%
    | \.\.\.                # ellipsis
    | [][.,;"'?():-_`]      # these are separate tokens
'''

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

#Taken from Su Nam Kim Paper...
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""
chunker = nltk.RegexpParser(grammar)

toks = nltk.regexp_tokenize(sentence, sentence_re)
postoks = nltk.tag.pos_tag(tokens)
tree = chunker.parse(postoks)

from nltk.corpus import stopwords
stopwords = stopwords.words('english')


In [39]:
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

In [40]:
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word)
    return word

In [41]:
def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


In [42]:
def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term
        

In [43]:
terms = get_terms(tree)

In [44]:
for term in terms:
    for word in term:
        print word,
    print

man
woman
kitchen


In [ ]:
entities

In [ ]:
tree

In [ ]:
 rd_parser = nltk.RecursiveDescentParser(grammar)

In [ ]:
sent = 'mary saw a dog'.split()
for tr in rd_parser.parse(sent):
    print(tr)

In [2]:
from spacy.en import English
parser = English()
usnt = unicode('I shot an elephant')
#usnt = unicode(sentence)
parsedData=parser(usnt)

sub_toks = [tok for tok in parsedData if (tok.dep_ == "nsubj") ]
print sub_toks
for i, token1 in enumerate(parsedData):
    print("lemma:", token1, token1.dep_)

[I]
('lemma:', I, u'nsubj')
('lemma:', shot, u'ROOT')
('lemma:', an, u'det')
('lemma:', elephant, u'dobj')


In [3]:
example = unicode("The boy with the spotted dog quickly ran after the firetruck.")
parsedEx = parser(unicode(sentence))
for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])

NameError: name 'sentence' is not defined

In [59]:
# Let's see if it can figure out this analogy
# Man is to King as Woman is to ??
king = parser.vocab[unicode('king')]
man = parser.vocab[unicode('man')]
woman = parser.vocab[unicode('woman')]

result = king.vector - man.vector + woman.vector

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "king" and w.lower_ != "man" and w.lower_ != "woman"})
# sort by similarity to the result
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 closest results for king - man + woman:")
for word in allWords[:3]:   
    print(word.orth_)
    
# it got it! Queen!


----------------------------
Top 3 closest results for king - man + woman:
queen
prince
kings


In [4]:
from numpy import dot
from numpy.linalg import norm

# you can access known words from the parser's vocabulary
nasa = parser.vocab[unicode('ugly')]

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "nasa"})

# sort by similarity to NASA
allWords.sort(key=lambda w: cosine(w.vector, nasa.vector))
allWords.reverse()
print("Top 10 most similar words to NASA:")
for word in allWords[:10]:   
    print(word.orth_)

Top 10 most similar words to NASA:
ugly
hideous
awful
fugly
horrible
disgusting
stupid
unattractive
nasty
horrid


In [17]:
# Let's see if it can figure out this analogy
# Man is to King as Woman is to ??
king = parser.vocab[unicode('ugly')]
man = parser.vocab[unicode('man')]
woman = parser.vocab[unicode('woman')]

#result = king.vector*man.vector
result = woman.vector+man.vector

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "king" and w.lower_ != "man" and w.lower_ != "woman"})
# sort by similarity to the result
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 closest results for king - man + woman:")
for word in allWords[:100]:   
    print(word.orth_)
    
# it got it! Queen!


----------------------------
Top 3 closest results for king - man + woman:
lady
girl
boy
men
guy
young
mother
she
women
gentleman
wife
he
teenager
husband
father
stranger
person
someone
female
who
victim
male
sex
lover
him
himself
herself
sexy
girlfriend
her
teenage
knows
dead
one
friend
when
gay
wants
mom
thing
girls
policeman
life
dude
cop
mans
boyfriend
soldier
never
pregnant
ladies
dad
drunk
knew
naked
being
ever
nothing
somebody
suspect
kind
his
fuck
say
teen
lad
face
whom
brother
prostitute
lives
thought
fucking
mind
told
that
way
alone
sexual
apparently
raped
looking
tells
housewife
horny
daughter
killed
lonely
tell
finds
blonde
boys
was
telling
innocent
wonder
once
death
couple
story
